In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("data/atp_matches_1990.csv")

In [4]:
y = 1991

In [5]:
while y < 2025:
    temp = pd.read_csv(f"data/atp_matches_{y}.csv")
    data = pd.concat([data, temp], ignore_index=True)
    y += 1

In [6]:
df = data

In [7]:
new_columns = {col: col.replace('winner', 'w') for col in df.columns if col.startswith('winner')}


In [8]:
new_columns

{'winner_id': 'w_id',
 'winner_seed': 'w_seed',
 'winner_entry': 'w_entry',
 'winner_name': 'w_name',
 'winner_hand': 'w_hand',
 'winner_ht': 'w_ht',
 'winner_ioc': 'w_ioc',
 'winner_age': 'w_age',
 'winner_rank': 'w_rank',
 'winner_rank_points': 'w_rank_points'}

In [9]:
new_columns.update({col: col.replace('loser', 'l') for col in df.columns if col.startswith("loser")})

In [10]:
df = df.rename(columns=new_columns)

In [11]:
winner_cols = [col for col in df.columns if col.startswith('w')]
loser_cols = [col for col in df.columns if col.startswith('l')]
other_cols = [col for col in df.columns if col not in winner_cols and col not in loser_cols]

In [12]:
stat_names = [col.replace('w_', '') for col in winner_cols]

In [13]:
swap_mask = np.random.rand(len(df)) < 0.5

In [14]:
new_df = pd.DataFrame()

In [15]:
for stat in stat_names:
    winner_stat_col = f'w_{stat}'
    loser_stat_col = f'l_{stat}'
    
    new_df[f'p1_{stat}'] = np.where(swap_mask, df[loser_stat_col], df[winner_stat_col])
    
    new_df[f'p2_{stat}'] = np.where(swap_mask, df[winner_stat_col], df[loser_stat_col])

In [16]:
new_df['winner'] = np.where(swap_mask, 'p2', 'p1')

In [17]:
for col in other_cols:
    new_df[col] = df[col]

In [18]:
new_df['tourney_date'] = pd.to_datetime(new_df['tourney_date'], format='%Y%m%d')

In [21]:
players = pd.read_csv("data/atp_players.csv")

/var/folders/3m/83d_phts1yg9dsv29rxnq4n80000gn/T/ipykernel_35325/816518900.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("data/atp_players.csv")


In [23]:
players.head()

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554


In [24]:
new_df.head()

,p1_id,p2_id,p1_seed,p2_seed,p1_entry,p2_entry,p1_name,p2_name,p1_hand,p2_hand,...,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,score,best_of,round,minutes
0,101843,101632,1.0,NaN,NaN,Q,Sergi Bruguera,Per Henricsson,R,R,...,Adelaide,Hard,32,A,1990-01-01,1,7-6 6-2,3,R32,NaN
1,100714,101096,NaN,NaN,WC,NaN,Broderick Dyke,Gary Muller,L,L,...,Adelaide,Hard,32,A,1990-01-01,2,6-4 6-4,3,R32,NaN
2,101232,101529,NaN,NaN,NaN,NaN,Nicklas Kroon,Michael Stich,R,R,...,Adelaide,Hard,32,A,1990-01-01,3,6-4 6-3,3,R32,NaN
3,101434,100628,NaN,7.0,NaN,NaN,Petr Korda,Michiel Schapers,L,R,...,Adelaide,Hard,32,A,1990-01-01,4,6-3 6-4,3,R32,NaN
4,101404,101387,3.0,NaN,NaN,NaN,Thomas Muster,Claudio Pistolesi,L,R,...,Adelaide,Hard,32,A,1990-01-01,5,6-4 4-6 6-3,3,R32,NaN


In [25]:
new_df = new_df.sort_values(by='tourney_date').reset_index(drop=True)

In [26]:
p_ids = new_df[['p1_id', 'p2_id']].astype(str)
new_df['matchup_id'] = p_ids.min(axis=1) + '-' + p_ids.max(axis=1)

In [53]:
new_df['low_won'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] < new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] < new_df['p1_id']))).astype(int)
new_df['high_won'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] > new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] > new_df['p1_id']))).astype(int)

In [57]:
cumulative_wins = new_df.groupby('matchup_id')[['low_won', 'high_won']].cumsum()

In [ ]:
new_df['high_won'] = cumulative_wins['high_won']
new_df['low_won'] =  cumulative_wins['low_won']

In [86]:
new_df['low_won'] = new_df.groupby('matchup_id')['low_won'].shift(1).fillna(0).astype(int)
new_df['high_won'] = new_df.groupby('matchup_id')['high_won'].shift(1).fillna(0).astype(int)

In [ ]:
#djokovic 104925 nadal 104745

In [117]:
new_df = new_df.sort_values(by="tourney_date").reset_index(drop=True)

In [128]:
new_df['low_won_surface'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] < new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] < new_df['p1_id']))).astype(int)
new_df['high_won_surface'] = (((new_df['winner'] == 'p1') & (new_df['p1_id'] > new_df['p2_id'])) | ((new_df['winner'] == 'p2') & (new_df['p2_id'] > new_df['p1_id']))).astype(int)

In [129]:
cumulative_surface_wins = new_df.groupby(['matchup_id', "surface"])[['low_won_surface', 'high_won_surface']].cumsum()

In [130]:
new_df['high_won_surface'] = cumulative_surface_wins['high_won_surface']
new_df['low_won_surface'] =  cumulative_surface_wins['low_won_surface']

In [131]:
new_df['low_won_surface'] = new_df.groupby(['matchup_id', 'surface'])['low_won_surface'].shift(1).fillna(0).astype(int)
new_df['high_won_surface'] = new_df.groupby(['matchup_id', 'surface'])['high_won_surface'].shift(1).fillna(0).astype(int)

In [ ]:
# TODO write the ELO methods, one for win differential, one for k-score, one for update and one to initialize

In [ ]:
def update_elos():
    return